#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [21]:
from etl import preprocess

In [24]:
# Import Queries
from sql_queries import session_item_create, user_session_create, song_user_create
from sql_queries import drop_table_queries

ImportError: cannot import name 'session_item_create' from 'sql_queries' (/Users/keneudeh/Documents/Projects/udacity-data-engineer-nanodegree/0-Data-Modeling/Projects/sparkify-data-etl-cassandra/sql_queries.py)

In [3]:
from utils import execute_query

## Run ETL Pipeline for Pre-Processing the Files

In [23]:
preprocess()

/Users/keneudeh/Documents/Projects/udacity-data-engineer-nanodegree/0-Data-Modeling/Projects/sparkify-data-etl-cassandra


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster

try:
    
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set Keyspace
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [10]:
execute_query(session_item_create)

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item (session_id, session_item_id, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
try:
    results = session.execute("""
    SELECT * FROM session_item
    WHERE session_id=338 AND session_item_id=4
    """)
except Exception as e:
    print(e)
 
for row in results:
    print('| {} | {} | {} | {} | {} | '.format(row.session_id, row.session_item_id, row.artist_name, row.song_title, row.song_length))

| 338 | 4 | Faithless | Music Matters (Mark Knight Dub) | 495.3073 | 


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
execute_query(user_session_create)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (user_id, session_id, session_item_id, artist_name, first_name, last_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
try:
    results = session.execute("""
    SELECT * FROM user_session
    WHERE user_id=10 AND session_id=182
    """)
except Exception as e:
    print(e)
 
for row in results:
    print('| {} | {} | {} | {} | {} | {} | {} |'.format(row.user_id , row.session_id, row.session_item_id, row.artist_name, 
                                                        row.first_name, row.last_name, row.song_title))

| 10 | 182 | 0 | Down To The Bone | Sylvie | Cruz | Keep On Keepin' On |
| 10 | 182 | 1 | Three Drives | Sylvie | Cruz | Greece 2000 |
| 10 | 182 | 2 | Sebastien Tellier | Sylvie | Cruz | Kilometer |
| 10 | 182 | 3 | Lonnie Gordon | Sylvie | Cruz | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
execute_query(song_user_create)            

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user (song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
try:
    results = session.execute("""
    SELECT * FROM song_user
    WHERE song_title='All Hands Against His Own'
    """)
except Exception as e:
    print(e)

print(type(results))
 
for row in results:
    print('| {} | {} | {} |'.format(row.song_title, row.first_name, row.last_name))

<class 'cassandra.cluster.ResultSet'>
| All Hands Against His Own | Jacqueline | Lynch |


### Drop the tables before closing out the sessions

In [19]:
for query in drop_table_queries:
    drop_collection(session, query)

NameError: name 'drop_collection' is not defined

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()